In [23]:
#Imports
import pandas as pd
from pathlib import Path

#Config / Settings
BASE = Path(r"C:\Users\damen\Desktop\football-main")

PROJ_DIR = BASE / "Projections"
ACT_DIRS = {
    'QB': BASE / r"Actuals\QB Stats",
    'RB': BASE / r"Actuals\RB Stats",
    'WR': BASE / r"Actuals\WR Stats",
    'TE': BASE / r"Actuals\TE Stats",
}

YEARS = range(2016, 2025)  # include 2016–2024
POSITIONS = ['QB', 'RB', 'WR', 'TE']

# Season game counts
SEASON_GAMES = {
    2016: 16, 2017: 16, 2018: 16, 2019: 16, 2020: 16,
    2021: 17, 2022: 17, 2023: 17, 2024: 17
}

# Projections loader
def load_projections(position: str, years=YEARS) -> pd.DataFrame:
    frames = []
    for y in years:
        df = pd.read_csv(PROJ_DIR / f"projections_{y}_wk0.csv")[['player','position','points']].copy()
        df['season'] = y
        df['player'] = df['player'].astype(str).str.strip()
        df = df[df['position'].str.upper() == position]
        frames.append(df)
    return pd.concat(frames, ignore_index=True)

# Actuals loader
def load_actuals(position: str, years=YEARS) -> pd.DataFrame:
    frames = []
    for y in years:
        df = pd.read_csv(ACT_DIRS[position] / f"{y} {position} Stats", sep=None, engine='python')
        df = df.rename(columns={'Player':'player'})[['player','FPTS']].copy()
        df['player'] = (
            df['player'].astype(str)
            .str.replace(r"\s*\([^)]*\)$", "", regex=True)  # strip (TEAM)
            .str.strip()
        )
        df['season'] = y
        df['position'] = position
        frames.append(df)
    return pd.concat(frames, ignore_index=True)

# Panel builder (all years for one position)
def build_panel(position: str) -> pd.DataFrame:
    proj = load_projections(position)
    act = load_actuals(position)
    panel = (
        proj.merge(
            act[['player','season','FPTS']],
            on=['player','season'],
            how='inner'
        )
        .rename(columns={'points':'proj_points','FPTS':'actual_FPTS'})
    )
    panel['error'] = panel['proj_points'] - panel['actual_FPTS']
    panel['error_pct'] = panel['error'] / panel['proj_points']
    return panel

# Stack all positions + years
def build_all_panels(positions=POSITIONS) -> pd.DataFrame:
    return pd.concat([build_panel(pos) for pos in positions], ignore_index=True)

# ---- RUN IT ----
all_data = build_all_panels()
all_data.head()

,player,position,proj_points,season,actual_FPTS,error,error_pct
0,Cam Newton,QB,391.0,2016,254.3,136.7,0.349616
1,Aaron Rodgers,QB,386.0,2016,380.0,6.0,0.015544
2,Russell Wilson,QB,374.0,2016,270.1,103.9,0.277807
3,Andrew Luck,QB,365.0,2016,307.6,57.4,0.157260
4,Drew Brees,QB,352.0,2016,332.3,19.7,0.055966


In [24]:
# Position-level metrics from historical stacked data (all_data)
pos_metrics = (
    all_data
    .groupby('position')
    .agg(
        avg_proj=('proj_points','mean'),
        avg_actual=('actual_FPTS','mean'),
        bias=('error','mean'),                       # systematic over/under
        abs_miss=('error', lambda x: x.abs().mean()),# typical miss size
        error_std=('error','std')                    # volatility of misses
    )
    .reset_index()
)

# Reliability: lower spread relative to projection = more reliable
pos_metrics['reliability'] = 1 - (pos_metrics['error_std'] / pos_metrics['avg_proj'])
pos_metrics

,position,avg_proj,avg_actual,bias,abs_miss,error_std,reliability
0,QB,288.465385,218.621474,69.843910,90.697756,90.385964,0.686666
1,RB,159.308084,135.142531,24.165554,59.709315,73.546528,0.538338
2,TE,136.419136,112.951543,23.467593,46.611420,54.053487,0.603769
3,WR,197.449735,166.252205,31.197531,57.628219,67.294367,0.659182


In [25]:
# Load current-season projections (2025 if available, else 2024)
for y in (2025, 2024):
    try:
        proj_current = pd.read_csv(PROJ_DIR / f"projections_{y}_wk0.csv")[['player','position','points']].copy()
        proj_current['season'] = y
        CURRENT_YEAR = y
        break
    except FileNotFoundError:
        continue

print(f"Using projections for {CURRENT_YEAR}")
proj_current.head()

Using projections for 2025


,player,position,points,season
0,Bijan Robinson,RB,332.0,2025
1,Saquon Barkley,RB,319.0,2025
2,Jahmyr Gibbs,RB,318.0,2025
3,Ja'Marr Chase,WR,355.0,2025
4,Lamar Jackson,QB,438.0,2025


In [26]:
# Merge position-level bias/reliability/abs_miss into current projections
proj_current = proj_current.merge(
    pos_metrics[['position','bias','reliability','abs_miss']],
    on='position', how='left'
)

# Adjust projections with historical bias & reliability
proj_current['adj_points']  = (proj_current['points'] - proj_current['bias']) * proj_current['reliability']

# Tier score: reward adjusted points; penalize historically noisy positions
proj_current['tier_score'] = proj_current['adj_points'] / (proj_current['abs_miss'] + 1e-6)

proj_current[['player','position','points','adj_points','tier_score']].head()

,player,position,points,adj_points,tier_score
0,Bijan Robinson,RB,332.0,165.718905,2.775428
1,Saquon Barkley,RB,319.0,158.720514,2.658220
2,Jahmyr Gibbs,RB,318.0,158.182177,2.649204
3,Ja'Marr Chase,WR,355.0,213.444853,3.703825
4,Lamar Jackson,QB,438.0,252.800332,2.787283


In [27]:
# Desired tiers per position
TIER_COUNTS = {'WR': 10, 'RB': 8, 'QB': 6, 'TE': 5}

def assign_tiers_by_position(df, tier_counts=TIER_COUNTS):
    out = []
    for pos, sub in df.groupby('position', group_keys=False):
        n = tier_counts.get(pos, 5)
        sub = sub.sort_values('tier_score', ascending=False).reset_index(drop=True)
        # percentile rank then cut into n bins (robust when few players)
        pr = sub.index / max(len(sub)-1, 1)
        sub['tier'] = pd.cut(pr, bins=n, labels=range(1, n+1), include_lowest=True)
        out.append(sub)
    return pd.concat(out, ignore_index=True)

proj_current_tiers = assign_tiers_by_position(proj_current)
proj_current_tiers[['player','position','points','adj_points','tier_score','tier']].head(20)

,player,position,points,adj_points,tier_score,tier
0,Lamar Jackson,QB,438.0,252.800332,2.787283,1
1,Josh Allen,QB,432.0,248.680335,2.741858,1
2,Joe Burrow,QB,415.0,237.007010,2.613152,1
3,Jayden Daniels,QB,412.0,234.947012,2.590439,1
4,Jalen Hurts,QB,398.0,225.333685,2.484446,1
5,Baker Mayfield,QB,377.0,210.913696,2.325457,1
6,Patrick Mahomes,QB,376.0,210.227029,2.317886,2
7,Kyler Murray,QB,361.0,199.927037,2.204322,2
8,Bo Nix,QB,356.0,196.493706,2.166467,2
9,Brock Purdy,QB,352.0,193.747041,2.136183,2


In [28]:
# Top 15 per position (adjusted)
def top_n(pos, n=15):
    cols = ['player','position','points','adj_points','tier_score','tier']
    return (proj_current_tiers.query("position == @pos")
            .sort_values('tier_score', ascending=False)
            [cols].head(n))

top_n('WR', 15)
# top_n('RB', 15); top_n('QB', 10); top_n('TE', 10)

,player,position,points,adj_points,tier_score,tier
144,Ja'Marr Chase,WR,355.0,213.444853,3.703825,1
145,Justin Jefferson,WR,317.0,188.395926,3.269161,1
146,CeeDee Lamb,WR,312.0,185.100015,3.211968,1
147,Puka Nacua,WR,305.0,180.485739,3.131899,1
148,Malik Nabers,WR,302.0,178.508192,3.097583,1
149,Amon-Ra St. Brown,WR,298.0,175.871462,3.051829,1
150,Nico Collins,WR,287.0,168.620457,2.926005,1
151,Brian Thomas Jr.,WR,284.0,166.642910,2.891689,1
152,Drake London,WR,268.0,156.095994,2.708673,2
153,A.J. Brown,WR,265.0,154.118447,2.674357,2


##### Option 1.A - Weight reliability more heavily

In [29]:
# Option A.1 - reliability weighted more heavily
proj_current['tier_score_A1'] = (
    (proj_current['points'] - proj_current['bias'])
    * (proj_current['reliability'] ** 2)   # heavier weight on reliability
    / (proj_current['abs_miss'] + 1e-6)
)

# Assign tiers for this option (pass only the right column)
proj_tiers_A1 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A1']].rename(columns={'tier_score_A1':'tier_score'})
)

# Preview top WRs under Option A.1
proj_tiers_A1.query("position == 'WR'")[['player','points','adj_points','tier_score','tier']].head(65)

,player,points,adj_points,tier_score,tier
144,Ja'Marr Chase,355.0,213.444853,2.441496,1
145,Justin Jefferson,317.0,188.395926,2.154973,1
146,CeeDee Lamb,312.0,185.100015,2.117273,1
147,Puka Nacua,305.0,180.485739,2.064492,1
148,Malik Nabers,302.0,178.508192,2.041872,1
...,...,...,...,...,...
204,Amari Cooper,145.0,75.016572,0.858080,9
205,Xavier Legette,144.0,74.357390,0.850539,9
206,Demario Douglas,143.0,73.698208,0.842999,9
207,Marquise Brown,142.0,73.039025,0.835459,9


In [30]:
def show_tiers_multi(df: pd.DataFrame, position: str, n: int = 30, per_col: int = 15):
    """
    Show top n players for a given position in multiple columns.
    Each column has `per_col` rows. Handles uneven last column.
    """
    # Get sorted slice (tier first, then score/points)
    table = (
        df.loc[df['position'] == position, ['player','points','tier_score','tier']]
          .sort_values(by=['tier','tier_score'], ascending=[True, False])
          .head(n)
          .reset_index(drop=True)
    )
    
    # Split into chunks
    chunks = [table.iloc[i:i+per_col].reset_index(drop=True) 
              for i in range(0, len(table), per_col)]
    
    # Rename columns for each chunk
    renamed = []
    for idx, c in enumerate(chunks, 1):
        c.columns = [f'player_{idx}', f'points_{idx}', f'tier_score_{idx}', f'tier_{idx}']
        renamed.append(c)
    
    return pd.concat(renamed, axis=1)

# Option A.1 - heavier reliability weight
proj_current['tier_score_A1'] = (
    (proj_current['points'] - proj_current['bias'])
    * (proj_current['reliability'] ** 2)
    / (proj_current['abs_miss'] + 1e-6)
)

proj_tiers_A1 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A1']]
    .rename(columns={'tier_score_A1':'tier_score'})
)

# Example: top 72 WRs, 24 per column
show_tiers_multi(proj_tiers_A1, 'WR', n=72, per_col=24)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Ja'Marr Chase,355.0,2.441496,1,Zay Flowers,221.0,1.431125,4,Josh Downs,179.0,1.114442,7
1,Justin Jefferson,317.0,2.154973,1,Calvin Ridley,221.0,1.431125,4,Darnell Mooney,178.0,1.106902,7
2,CeeDee Lamb,312.0,2.117273,1,Jerry Jeudy,219.0,1.416045,4,Rashid Shaheed,172.0,1.061662,7
3,Puka Nacua,305.0,2.064492,1,Rashee Rice,215.0,1.385885,4,Keon Coleman,168.0,1.031501,8
4,Malik Nabers,302.0,2.041872,1,Jameson Williams,214.0,1.378345,4,Keenan Allen,161.0,0.978721,8
5,Amon-Ra St. Brown,298.0,2.011712,1,Jaylen Waddle,211.0,1.355725,5,Adam Thielen,160.0,0.971181,8
6,Nico Collins,287.0,1.928771,1,Jakobi Meyers,210.0,1.348185,5,Wan'Dale Robinson,157.0,0.948560,8
7,Brian Thomas Jr.,284.0,1.906150,1,George Pickens,209.0,1.340645,5,Rashod Bateman,153.0,0.918400,8
8,Drake London,268.0,1.785509,2,Travis Hunter,207.0,1.325564,5,Marvin Mims,153.0,0.918400,8
9,A.J. Brown,265.0,1.762889,2,Travis Hunter,207.0,1.325564,5,Emeka Egbuka,152.0,0.910860,8


In [31]:
def show_tiers_multi(df: pd.DataFrame, position: str, n: int = 30, per_col: int = 15):
    """
    Show top n players for a given position in multiple columns.
    Each column has `per_col` rows. Handles uneven last column.
    """
    # Get sorted slice (tier first, then score/points)
    table = (
        df.loc[df['position'] == position, ['player','points','tier_score','tier']]
          .sort_values(by=['tier','tier_score'], ascending=[True, False])
          .head(n)
          .reset_index(drop=True)
    )
    
    # Split into chunks
    chunks = [table.iloc[i:i+per_col].reset_index(drop=True) 
              for i in range(0, len(table), per_col)]
    
    # Rename columns for each chunk
    renamed = []
    for idx, c in enumerate(chunks, 1):
        c.columns = [f'player_{idx}', f'points_{idx}', f'tier_score_{idx}', f'tier_{idx}']
        renamed.append(c)
    
    return pd.concat(renamed, axis=1)

# Option A.1 - heavier reliability weight
proj_current['tier_score_A1'] = (
    (proj_current['points'] - proj_current['bias'])
    * (proj_current['reliability'] ** 2)
    / (proj_current['abs_miss'] + 1e-6)
)

proj_tiers_A1 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A1']]
    .rename(columns={'tier_score_A1':'tier_score'})
)

# Example: top 72 RBs, 24 per column
show_tiers_multi(proj_tiers_A1, 'RB', n=72, per_col=24)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Bijan Robinson,332.0,1.494118,1,David Montgomery,199.0,0.848584,3,Tyler Allgeier,102.0,0.377780,6
1,Saquon Barkley,319.0,1.431020,1,Joe Mixon,195.0,0.829169,3,Justice Hill,102.0,0.377780,6
2,Jahmyr Gibbs,318.0,1.426167,1,Tyrone Tracy,191.0,0.809754,3,Trey Benson,101.0,0.372927,6
3,Devon Achane,297.0,1.324240,1,Isiah Pacheco,180.0,0.756364,4,Isaac Guerendo,99.8,0.367102,6
4,Christian McCaffrey,288.0,1.280557,1,Jaylen Warren,176.0,0.736950,4,Kareem Hunt,97.4,0.355454,6
5,Derrick Henry,275.0,1.217460,1,Brian Robinson,173.0,0.722389,4,Roschon Johnson,95.5,0.346232,6
6,Josh Jacobs,275.0,1.217460,1,Kaleb Johnson,171.0,0.712682,4,Nick Chubb,93.6,0.337010,7
7,Ashton Jeanty,274.0,1.212607,1,Austin Ekeler,157.0,0.644731,4,Bhayshul Tuten,93.0,0.334098,7
8,Chase Brown,268.0,1.183485,1,Javonte Williams,155.0,0.635023,4,Braelon Allen,90.5,0.321964,7
9,Bucky Irving,265.0,1.168924,2,Rhamondre Stevenson,151.0,0.615609,4,Zack Moss,88.1,0.310315,7


In [32]:
def show_tiers_multi(df: pd.DataFrame, position: str, n: int = 30, per_col: int = 15):
    """
    Show top n players for a given position in multiple columns.
    Each column has `per_col` rows. Handles uneven last column.
    """
    # Get sorted slice (tier first, then score/points)
    table = (
        df.loc[df['position'] == position, ['player','points','tier_score','tier']]
          .sort_values(by=['tier','tier_score'], ascending=[True, False])
          .head(n)
          .reset_index(drop=True)
    )
    
    # Split into chunks
    chunks = [table.iloc[i:i+per_col].reset_index(drop=True) 
              for i in range(0, len(table), per_col)]
    
    # Rename columns for each chunk
    renamed = []
    for idx, c in enumerate(chunks, 1):
        c.columns = [f'player_{idx}', f'points_{idx}', f'tier_score_{idx}', f'tier_{idx}']
        renamed.append(c)
    
    return pd.concat(renamed, axis=1)

# Option A.1 - heavier reliability weight
proj_current['tier_score_A1'] = (
    (proj_current['points'] - proj_current['bias'])
    * (proj_current['reliability'] ** 2)
    / (proj_current['abs_miss'] + 1e-6)
)

proj_tiers_A1 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A1']]
    .rename(columns={'tier_score_A1':'tier_score'})
)

# Example: top 72 QBs, 24 per column
show_tiers_multi(proj_tiers_A1, 'QB', n=36, per_col=12)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Lamar Jackson,438.0,1.913933,1,Justin Herbert,338.0,1.394063,3,Michael Penix Jr.,298.0,1.186115,5
1,Josh Allen,432.0,1.882741,1,Dak Prescott,336.0,1.383666,3,Aaron Rodgers,293.0,1.160122,5
2,Joe Burrow,415.0,1.794363,1,Justin Fields,333.0,1.368070,3,Sam Darnold,290.0,1.144525,5
3,Jayden Daniels,412.0,1.778767,1,J.J. McCarthy,323.0,1.316083,3,Cam Ward,279.0,1.087340,5
4,Jalen Hurts,398.0,1.705985,1,Jordan Love,323.0,1.316083,3,Anthony Richardson,237.0,0.868994,5
5,Baker Mayfield,377.0,1.596812,1,C.J. Stroud,320.0,1.300486,3,Russell Wilson,223.0,0.796213,5
6,Patrick Mahomes,376.0,1.591614,2,Drake Maye,319.0,1.295288,4,Tyler Shough,168.0,0.510284,6
7,Kyler Murray,361.0,1.513633,2,Trevor Lawrence,318.0,1.290089,4,Daniel Jones,125.0,0.286740,6
8,Bo Nix,356.0,1.487640,2,Matthew Stafford,309.0,1.243301,4,Joe Flacco,118.0,0.250349,6
9,Brock Purdy,352.0,1.466845,2,Tua Tagovailoa,306.0,1.227705,4,Spencer Rattler,108.0,0.198362,6


In [33]:
def show_tiers_multi(df: pd.DataFrame, position: str, n: int = 30, per_col: int = 15):
    """
    Show top n players for a given position in multiple columns.
    Each column has `per_col` rows. Handles uneven last column.
    """
    # Get sorted slice (tier first, then score/points)
    table = (
        df.loc[df['position'] == position, ['player','points','tier_score','tier']]
          .sort_values(by=['tier','tier_score'], ascending=[True, False])
          .head(n)
          .reset_index(drop=True)
    )
    
    # Split into chunks
    chunks = [table.iloc[i:i+per_col].reset_index(drop=True) 
              for i in range(0, len(table), per_col)]
    
    # Rename columns for each chunk
    renamed = []
    for idx, c in enumerate(chunks, 1):
        c.columns = [f'player_{idx}', f'points_{idx}', f'tier_score_{idx}', f'tier_{idx}']
        renamed.append(c)
    
    return pd.concat(renamed, axis=1)

# Option A.1 - heavier reliability weight
proj_current['tier_score_A1'] = (
    (proj_current['points'] - proj_current['bias'])
    * (proj_current['reliability'] ** 2)
    / (proj_current['abs_miss'] + 1e-6)
)

proj_tiers_A1 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A1']]
    .rename(columns={'tier_score_A1':'tier_score'})
)

# Example: top 72 TEs, 24 per column
show_tiers_multi(proj_tiers_A1, 'TE', n=36, per_col=12)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Brock Bowers,259.0,1.842044,1,Dallas Goedert,152.0,1.005222,2,Mike Gesicki,130.0,0.833165,4
1,Trey McBride,254.0,1.802940,1,Jonnu Smith,147.0,0.966118,2,Dalton Schultz,129.0,0.825344,4
2,George Kittle,229.0,1.607421,1,Colston Loveland,147.0,0.966118,2,Isaiah Likely,127.0,0.809703,4
3,Sam LaPorta,195.0,1.341515,1,Zach Ertz,145.0,0.950477,3,Juwan Johnson,125.0,0.794061,4
4,Travis Kelce,185.0,1.263307,1,Hunter Henry,144.0,0.942656,3,Tyler Higbee,124.0,0.786240,4
5,T.J. Hockenson,183.0,1.247666,1,Kyle Pitts,141.0,0.919194,3,Theo Johnson,117.0,0.731495,5
6,Mark Andrews,180.0,1.224203,1,Dalton Kincaid,140.0,0.911373,3,Mason Taylor,116.0,0.723674,5
7,David Njoku,173.0,1.169458,1,Cade Otton,136.0,0.880090,3,Robbie Ouzts,113.0,0.700212,5
8,Evan Engram,164.0,1.099071,2,Darren Waller,133.0,0.856627,3,Elijah Arroyo,106.0,0.645467,5
9,Tucker Kraft,157.0,1.044326,2,Brenton Strange,133.0,0.856627,3,Ja'Tavion Sanders,97.0,0.575080,5


##### Option A.2 — normalize within position

In [34]:
# Option A.2 - normalize within position
proj_current['pos_mean'] = proj_current.groupby('position')['points'].transform('mean')

# % above positional mean using adjusted points
proj_current['adj_pct_above'] = (proj_current['adj_points'] - proj_current['pos_mean']) / proj_current['pos_mean']

proj_current['tier_score_A2'] = proj_current['adj_pct_above'] * proj_current['reliability']

In [35]:
proj_tiers_A2 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A2']]
    .rename(columns={'tier_score_A2':'tier_score'})
)

In [36]:
# Example: top 72 RBs, 24 per column
show_tiers_multi(proj_tiers_A2, 'RB', n=72, per_col=24)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Bijan Robinson,332.0,0.013588,1,David Montgomery,199.0,-0.224871,3,Tyler Allgeier,102.0,-0.398786,6
1,Saquon Barkley,319.0,-0.009720,1,Joe Mixon,195.0,-0.232043,3,Justice Hill,102.0,-0.398786,6
2,Jahmyr Gibbs,318.0,-0.011513,1,Tyrone Tracy,191.0,-0.239215,3,Trey Benson,101.0,-0.400579,6
3,Devon Achane,297.0,-0.049164,1,Isiah Pacheco,180.0,-0.258937,4,Isaac Guerendo,99.8,-0.402730,6
4,Christian McCaffrey,288.0,-0.065301,1,Jaylen Warren,176.0,-0.266109,4,Kareem Hunt,97.4,-0.407033,6
5,Derrick Henry,275.0,-0.088609,1,Brian Robinson,173.0,-0.271488,4,Roschon Johnson,95.5,-0.410440,6
6,Josh Jacobs,275.0,-0.088609,1,Kaleb Johnson,171.0,-0.275074,4,Nick Chubb,93.6,-0.413847,7
7,Ashton Jeanty,274.0,-0.090402,1,Austin Ekeler,157.0,-0.300175,4,Bhayshul Tuten,93.0,-0.414922,7
8,Chase Brown,268.0,-0.101159,1,Javonte Williams,155.0,-0.303760,4,Braelon Allen,90.5,-0.419405,7
9,Bucky Irving,265.0,-0.106538,2,Rhamondre Stevenson,151.0,-0.310932,4,Zack Moss,88.1,-0.423708,7


In [37]:
# Example: top 72 WRs, 24 per column
show_tiers_multi(proj_tiers_A2, 'WR', n=72, per_col=24)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Ja'Marr Chase,355.0,0.027659,1,Zay Flowers,221.0,-0.256578,4,Josh Downs,179.0,-0.345667,7
1,Justin Jefferson,317.0,-0.052945,1,Calvin Ridley,221.0,-0.256578,4,Darnell Mooney,178.0,-0.347789,7
2,CeeDee Lamb,312.0,-0.063551,1,Jerry Jeudy,219.0,-0.260820,4,Rashid Shaheed,172.0,-0.360516,7
3,Puka Nacua,305.0,-0.078399,1,Rashee Rice,215.0,-0.269305,4,Keon Coleman,168.0,-0.369000,8
4,Malik Nabers,302.0,-0.084763,1,Jameson Williams,214.0,-0.271426,4,Keenan Allen,161.0,-0.383849,8
5,Amon-Ra St. Brown,298.0,-0.093248,1,Jaylen Waddle,211.0,-0.277790,5,Adam Thielen,160.0,-0.385970,8
6,Nico Collins,287.0,-0.116580,1,Jakobi Meyers,210.0,-0.279911,5,Wan'Dale Robinson,157.0,-0.392333,8
7,Brian Thomas Jr.,284.0,-0.122944,1,George Pickens,209.0,-0.282032,5,Rashod Bateman,153.0,-0.400818,8
8,Drake London,268.0,-0.156883,2,Travis Hunter,207.0,-0.286274,5,Marvin Mims,153.0,-0.400818,8
9,A.J. Brown,265.0,-0.163246,2,Travis Hunter,207.0,-0.286274,5,Emeka Egbuka,152.0,-0.402939,8


In [38]:
# Example: top 72 WRs, 24 per column
show_tiers_multi(proj_tiers_A2, 'QB', n=36, per_col=12)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Lamar Jackson,438.0,-0.105300,1,Justin Herbert,338.0,-0.263213,3,Michael Penix Jr.,298.0,-0.326378,5
1,Josh Allen,432.0,-0.114775,1,Dak Prescott,336.0,-0.266371,3,Aaron Rodgers,293.0,-0.334274,5
2,Joe Burrow,415.0,-0.141620,1,Justin Fields,333.0,-0.271109,3,Sam Darnold,290.0,-0.339011,5
3,Jayden Daniels,412.0,-0.146358,1,J.J. McCarthy,323.0,-0.286900,3,Cam Ward,279.0,-0.356382,5
4,Jalen Hurts,398.0,-0.168465,1,Jordan Love,323.0,-0.286900,3,Anthony Richardson,237.0,-0.422705,5
5,Baker Mayfield,377.0,-0.201627,1,C.J. Stroud,320.0,-0.291637,3,Russell Wilson,223.0,-0.444813,5
6,Patrick Mahomes,376.0,-0.203206,2,Drake Maye,319.0,-0.293217,4,Tyler Shough,168.0,-0.531665,6
7,Kyler Murray,361.0,-0.226893,2,Trevor Lawrence,318.0,-0.294796,4,Daniel Jones,125.0,-0.599568,6
8,Bo Nix,356.0,-0.234789,2,Matthew Stafford,309.0,-0.309008,4,Joe Flacco,118.0,-0.610621,6
9,Brock Purdy,352.0,-0.241105,2,Tua Tagovailoa,306.0,-0.313745,4,Spencer Rattler,108.0,-0.626413,6


In [39]:
# Example: top 72 WRs, 24 per column
show_tiers_multi(proj_tiers_A2, 'TE', n=36, per_col=12)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Brock Bowers,259.0,-0.024925,1,Dallas Goedert,152.0,-0.287888,2,Mike Gesicki,130.0,-0.341955,4
1,Trey McBride,254.0,-0.037213,1,Jonnu Smith,147.0,-0.300176,2,Dalton Schultz,129.0,-0.344413,4
2,George Kittle,229.0,-0.098653,1,Colston Loveland,147.0,-0.300176,2,Isaiah Likely,127.0,-0.349328,4
3,Sam LaPorta,195.0,-0.182211,1,Zach Ertz,145.0,-0.305091,3,Juwan Johnson,125.0,-0.354243,4
4,Travis Kelce,185.0,-0.206787,1,Hunter Henry,144.0,-0.307549,3,Tyler Higbee,124.0,-0.356701,4
5,T.J. Hockenson,183.0,-0.211702,1,Kyle Pitts,141.0,-0.314921,3,Theo Johnson,117.0,-0.373904,5
6,Mark Andrews,180.0,-0.219075,1,Dalton Kincaid,140.0,-0.317379,3,Mason Taylor,116.0,-0.376361,5
7,David Njoku,173.0,-0.236278,1,Cade Otton,136.0,-0.327209,3,Robbie Ouzts,113.0,-0.383734,5
8,Evan Engram,164.0,-0.258397,2,Darren Waller,133.0,-0.334582,3,Elijah Arroyo,106.0,-0.400937,5
9,Tucker Kraft,157.0,-0.275600,2,Brenton Strange,133.0,-0.334582,3,Ja'Tavion Sanders,97.0,-0.423056,5


In [40]:
# Option A.3 - add uncertainty bands
proj_current['floor_A3']   = proj_current['adj_points'] - proj_current['abs_miss']
proj_current['ceiling_A3'] = proj_current['adj_points'] + proj_current['abs_miss']

# Floor-based score
proj_current['tier_score_A3_floor'] = proj_current['floor_A3'] * proj_current['reliability']

# Ceiling-based score
proj_current['tier_score_A3_ceiling'] = proj_current['ceiling_A3'] * proj_current['reliability']

# Floor tiers
proj_tiers_A3_floor = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A3_floor']]
    .rename(columns={'tier_score_A3_floor':'tier_score'})
)

# Ceiling tiers
proj_tiers_A3_ceiling = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A3_ceiling']]
    .rename(columns={'tier_score_A3_ceiling':'tier_score'})
)

In [41]:
# Top 60 WRs by floor (safe)
show_tiers_multi(proj_tiers_A3_floor, 'WR', n=60, per_col=20)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Ja'Marr Chase,355.0,102.711566,1,DK Metcalf,233.0,49.699968,3,Stefon Diggs,196.0,33.622680,6
1,Justin Jefferson,317.0,86.199757,1,Courtland Sutton,230.0,48.396404,3,Khalil Shakir,191.0,31.450074,6
2,CeeDee Lamb,312.0,84.027150,1,DeVonta Smith,229.0,47.961883,4,Cooper Kupp,190.0,31.015553,6
3,Puka Nacua,305.0,80.985501,1,Xavier Worthy,226.0,46.658319,4,Jayden Reed,189.0,30.581031,6
4,Malik Nabers,302.0,79.681937,1,Zay Flowers,221.0,44.485713,4,Jordan Addison,189.0,30.581031,7
5,Amon-Ra St. Brown,298.0,77.943852,1,Calvin Ridley,221.0,44.485713,4,Matthew Golden,187.0,29.711989,7
6,Nico Collins,287.0,73.164118,1,Jerry Jeudy,219.0,43.616670,4,Michael Pittman,185.0,28.842946,7
7,Brian Thomas Jr.,284.0,71.860554,1,Rashee Rice,215.0,41.878585,4,Ricky Pearsall,185.0,28.842946,7
8,Drake London,268.0,64.908213,2,Jameson Williams,214.0,41.444064,4,Josh Downs,179.0,26.235819,7
9,A.J. Brown,265.0,63.604650,2,Jaylen Waddle,211.0,40.140500,5,Darnell Mooney,178.0,25.801297,7


In [42]:
# Top 60 WRs by ceiling (upside)
show_tiers_multi(proj_tiers_A3_ceiling, 'WR', n=60, per_col=20)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Ja'Marr Chase,355.0,178.686568,1,DK Metcalf,233.0,125.674971,3,Stefon Diggs,196.0,109.597683,6
1,Justin Jefferson,317.0,162.174759,1,Courtland Sutton,230.0,124.371407,3,Khalil Shakir,191.0,107.425076,6
2,CeeDee Lamb,312.0,160.002153,1,DeVonta Smith,229.0,123.936885,4,Cooper Kupp,190.0,106.990555,6
3,Puka Nacua,305.0,156.960504,1,Xavier Worthy,226.0,122.633322,4,Jayden Reed,189.0,106.556034,6
4,Malik Nabers,302.0,155.656940,1,Zay Flowers,221.0,120.460715,4,Jordan Addison,189.0,106.556034,7
5,Amon-Ra St. Brown,298.0,153.918854,1,Calvin Ridley,221.0,120.460715,4,Matthew Golden,187.0,105.686991,7
6,Nico Collins,287.0,149.139120,1,Jerry Jeudy,219.0,119.591672,4,Michael Pittman,185.0,104.817949,7
7,Brian Thomas Jr.,284.0,147.835556,1,Rashee Rice,215.0,117.853587,4,Ricky Pearsall,185.0,104.817949,7
8,Drake London,268.0,140.883216,2,Jameson Williams,214.0,117.419066,4,Josh Downs,179.0,102.210821,7
9,A.J. Brown,265.0,139.579652,2,Jaylen Waddle,211.0,116.115502,5,Darnell Mooney,178.0,101.776300,7


In [43]:
# Weights for blending (you can tune these)
w_bias    = 1.0    # how strongly to correct for bias
w_rel     = 2.0    # reliability weight (use squared)
w_norm    = 1.0    # how much to scale by % above position mean
w_floor   = 0.5    # weight on floor (safety)
w_ceiling = 0.5    # weight on ceiling (upside)

# Normalization already computed in A.2
proj_current['pos_mean'] = proj_current.groupby('position')['points'].transform('mean')
proj_current['adj_pct_above'] = (proj_current['adj_points'] - proj_current['pos_mean']) / proj_current['pos_mean']

# Floor & ceiling already computed in A.3
proj_current['floor_A3']   = proj_current['adj_points'] - proj_current['abs_miss']
proj_current['ceiling_A3'] = proj_current['adj_points'] + proj_current['abs_miss']

# Fused score
proj_current['tier_score_A4'] = (
    ((proj_current['points'] - w_bias * proj_current['bias']) *
     (proj_current['reliability'] ** w_rel)) +
    (w_norm * proj_current['adj_pct_above'] * 100) +         # scale to %
    (w_floor * proj_current['floor_A3']) +
    (w_ceiling * proj_current['ceiling_A3'])
)

In [44]:
proj_tiers_A4 = assign_tiers_by_position(
    proj_current[['player','position','points','adj_points','tier_score_A4']]
    .rename(columns={'tier_score_A4':'tier_score'})
)

In [45]:
# Example: top 72 WRs, 24 per column under fused A.4
show_tiers_multi(proj_tiers_A4, 'WR', n=72, per_col=24)

,player_1,points_1,tier_score_1,tier_1,player_2,points_2,tier_score_2,tier_2,player_3,points_3,tier_score_3,tier_3
0,Ja'Marr Chase,355.0,358.339950,1,Zay Flowers,221.0,168.663962,4,Josh Downs,179.0,109.213279,7
1,Justin Jefferson,317.0,304.551237,1,Calvin Ridley,221.0,168.663962,4,Darnell Mooney,178.0,107.797786,7
2,CeeDee Lamb,312.0,297.473774,1,Jerry Jeudy,219.0,165.832977,4,Rashid Shaheed,172.0,99.304832,7
3,Puka Nacua,305.0,287.565327,1,Rashee Rice,215.0,160.171007,4,Keon Coleman,168.0,93.642862,8
4,Malik Nabers,302.0,283.318850,1,Jameson Williams,214.0,158.755514,4,Keenan Allen,161.0,83.734415,8
5,Amon-Ra St. Brown,298.0,277.656880,1,Jaylen Waddle,211.0,154.509037,5,Adam Thielen,160.0,82.318922,8
6,Nico Collins,287.0,262.086463,1,Jakobi Meyers,210.0,153.093545,5,Wan'Dale Robinson,157.0,78.072445,8
7,Brian Thomas Jr.,284.0,257.839986,1,George Pickens,209.0,151.678052,5,Rashod Bateman,153.0,72.410475,8
8,Drake London,268.0,235.192107,2,Travis Hunter,207.0,148.847067,5,Marvin Mims,153.0,72.410475,8
9,A.J. Brown,265.0,230.945629,2,Travis Hunter,207.0,148.847067,5,Emeka Egbuka,152.0,70.994983,8


In [47]:
# Sort ALL players across positions by fused tier_score (Option A.4)
sorted_all = (
    proj_tiers_A4
    .sort_values(by='tier_score', ascending=False)
    .reset_index(drop=True)
)

# Add an overall rank column
sorted_all['overall_rank'] = range(1, len(sorted_all) + 1)

# Preview top 50 overall
sorted_all[['overall_rank','player','position','points','adj_points','tier_score','tier']].head(50)

,overall_rank,player,position,points,adj_points,tier_score,tier
0,1,Lamar Jackson,QB,438.0,252.800332,411.054784,1
1,2,Josh Allen,QB,432.0,248.680335,402.725902,1
2,3,Joe Burrow,QB,415.0,237.007010,379.127402,1
3,4,Jayden Daniels,QB,412.0,234.947012,374.962961,1
4,5,Ja'Marr Chase,WR,355.0,213.444853,358.339950,1
5,6,Jalen Hurts,QB,398.0,225.333685,355.528903,1
6,7,Baker Mayfield,QB,377.0,210.913696,326.377815,1
7,8,Patrick Mahomes,QB,376.0,210.227029,324.989668,2
8,9,Justin Jefferson,WR,317.0,188.395926,304.551237,1
9,10,Kyler Murray,QB,361.0,199.927037,304.167462,2
